# lenet_mnist

In [1]:
import mxnet as mx
import os
import struct
import numpy as np
import logging
import random
logging.getLogger().setLevel(logging.DEBUG)

In [2]:
def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,'{}-labels-idx1-ubyte'.format(kind))
    images_path = os.path.join(path,'{}-images-idx3-ubyte'.format(kind))
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II',
                                 lbpath.read(8))
        labels = np.fromfile(lbpath,
                             dtype=np.uint8).reshape(n)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack('>IIII',
                                               imgpath.read(16))
        images = np.fromfile(imgpath,
                             dtype=np.uint8).reshape((num,1,rows,cols))
    print(kind)
    print("label num:",n)
    print("image num:",num)
    print("image rows:",rows)
    print("image cols:",cols)
    images = images/255
    return images, labels

In [3]:
'''
conv(5,5,6) relu avg_pool
conv(5,5,16) relu avg_pool
fc 120 relu
fc 84 relu
 softmax output
'''
def lenet5_v1(data):
    C1 = mx.sym.Convolution(data=data,kernel=(5,5),num_filter=6,name='C1')
    C1 = mx.sym.Activation(data=C1,act_type='relu',name='C1_relu')
    S2 = mx.sym.Pooling(data=C1,kernel=(2,2),pool_type='avg',stride=(2,2),name='S2')
    C3 = mx.sym.Convolution(data=S2,kernel=(5,5),num_filter=16,name='C3')
    C3 = mx.sym.Activation(data=C3,act_type='relu',name='C3_relu')
    S4 = mx.sym.Pooling(data=C3,kernel=(2,2),pool_type='avg',stride=(2,2),name='S4')
    F5 = mx.sym.Convolution(data=S4,kernel=(5,5),num_filter=120,name='F5')
    F5 = mx.sym.Activation(data=F5,act_type='relu',name='F5_relu')
    F6 = mx.sym.FullyConnected(data=F5,num_hidden=84,name='F6')
    F6 = mx.sym.Activation(data=F6,act_type='relu',name='F6_relu')
    output = mx.sym.FullyConnected(data=F6,num_hidden=10,name="output")
    output = mx.sym.SoftmaxOutput(data=F6,name='softmax')
    return output

'''
conv(5,5,6) relu avg_pool
conv(5,5,16) relu avg_pool
conv(5,5,120) relu
fc 84 relu
fc 10 softmax
'''
def lenet5_v2(data):
    C1 = mx.sym.Convolution(data=data,kernel=(5,5),num_filter=6,name='C1')
    C1 = mx.sym.Activation(data=C1,act_type='relu',name='C1_relu')
    S2 = mx.sym.Pooling(data=C1,kernel=(2,2),pool_type='avg',stride=(2,2),name='S2')
    C3 = mx.sym.Convolution(data=S2,kernel=(5,5),num_filter=16,name='C3')
    C3 = mx.sym.Activation(data=C3,act_type='relu',name='C3_relu')
    S4 = mx.sym.Pooling(data=C3,kernel=(2,2),pool_type='avg',stride=(2,2),name='S4')
    C5 = mx.sym.Convolution(data=S4,kernel=(5,5),num_filter=120,name='C5')
    C5 = mx.sym.Activation(data=C5,act_type='relu',name='C5_relu')
    F6 = mx.sym.FullyConnected(data=C5,num_hidden=84,name='F6')
    F6 = mx.sym.Activation(data=F6,act_type='relu',name='F6_relu')
    output = mx.sym.FullyConnected(data=F6,num_hidden=10,name='output')
    output = mx.sym.SoftmaxOutput(data=output,name="softmax")
    return output

'''
conv(5,5,32) relu avg_pool
conv(5,5,64) relu avg_pool
fc 1024 relu
fc 10 softmax
'''
def lenet5_v3(data):
    C1 = mx.sym.Convolution(data=data,kernel=(5,5),num_filter=32,name='C1')
    C1 = mx.sym.Activation(data=C1,act_type='relu',name='C1_relu')
    S2 = mx.sym.Pooling(data=C1,kernel=(2,2),pool_type='avg',stride=(2,2),name='S2')
    C3 = mx.sym.Convolution(data=S2,kernel=(5,5),num_filter=64,name='C3')
    C3 = mx.sym.Activation(data=C3,act_type='relu',name='C3_relu')
    S4 = mx.sym.Pooling(data=C3,kernel=(2,2),pool_type='avg',stride=(2,2),name='S4')
    F5 = mx.sym.FullyConnected(data=S4,num_hidden=1024,name='F5')
    F5 = mx.sym.Activation(data=F5,act_type='relu',name='F5_relu')
    output = mx.sym.FullyConnected(data=F5,num_hidden=10,name='output')
    output = mx.sym.SoftmaxOutput(data=output,name='softmax')
    return output

In [4]:
BACH_SIZE = 1000
LEARNING_RATE = 0.1
MOMENTUM = 0.9

In [5]:
train_im,train_la = load_mnist('../dataset/mnist/','train')
val_im,val_la = load_mnist('../dataset/mnist/','t10k')

train
label num: 60000
image num: 60000
image rows: 28
image cols: 28
t10k
label num: 10000
image num: 10000
image rows: 28
image cols: 28


In [6]:
im_data = mx.sym.Variable(name='im_data')
net = lenet5_v3(im_data)

In [7]:
mod = mx.mod.Module(symbol=net,
                    context=mx.cpu(),
                    data_names=['im_data'],
                    label_names=['softmax_label'])

In [8]:
train_iter = mx.io.NDArrayIter(data=train_im,
                               label=train_la,
                               batch_size=BACH_SIZE,
                               data_name='im_data',
                               label_name='softmax_label')
val_iter = mx.io.NDArrayIter(data=val_im,
                             label=val_la,
                             batch_size=BACH_SIZE,
                            data_name='im_data',
                               label_name='softmax_label')

In [9]:
lr_factor = 0.1
steps = 300 # in iterations
lr_scheduler = mx.lr_scheduler.FactorScheduler(step=steps, factor=lr_factor)
optimizer_params = {
                    'learning_rate': LEARNING_RATE,
                    'wd': 0.0005,
                    'lr_scheduler': lr_scheduler,
                    'momentum': MOMENTUM}
initializer = mx.init.Xavier(
   rnd_type='gaussian', factor_type='in', magnitude=2)

In [10]:
speedometer = mx.callback.Speedometer(BACH_SIZE, 10)
progress_bar = mx.callback.ProgressBar(total=2)
logmc = mx.callback.LogValidationMetricsCallback()
train_log = mx.callback.log_train_metric(10)

In [11]:
mod.fit(train_iter,
        eval_data=val_iter,
        optimizer='sgd',
        optimizer_params=optimizer_params,
#         batch_end_callback=[speedometer],
        initializer = initializer,
        eval_metric='acc',
        num_epoch=15)

INFO:root:Epoch[0] Train-accuracy=0.849950
INFO:root:Epoch[0] Time cost=67.776
INFO:root:Epoch[0] Validation-accuracy=0.968900
INFO:root:Epoch[1] Train-accuracy=0.973883
INFO:root:Epoch[1] Time cost=69.866
INFO:root:Epoch[1] Validation-accuracy=0.982200
INFO:root:Epoch[2] Train-accuracy=0.982267
INFO:root:Epoch[2] Time cost=71.377
INFO:root:Epoch[2] Validation-accuracy=0.984500
INFO:root:Epoch[3] Train-accuracy=0.985950
INFO:root:Epoch[3] Time cost=69.820
INFO:root:Epoch[3] Validation-accuracy=0.985000
INFO:root:Epoch[4] Train-accuracy=0.988133
INFO:root:Epoch[4] Time cost=70.544
INFO:root:Epoch[4] Validation-accuracy=0.985100
INFO:root:Update[301]: Change learning rate to 1.00000e-02
INFO:root:Epoch[5] Train-accuracy=0.989050
INFO:root:Epoch[5] Time cost=70.009
INFO:root:Epoch[5] Validation-accuracy=0.988400
INFO:root:Epoch[6] Train-accuracy=0.991550
INFO:root:Epoch[6] Time cost=67.761
INFO:root:Epoch[6] Validation-accuracy=0.989000
INFO:root:Epoch[7] Train-accuracy=0.991767
INFO:root

# mxnet test

In [ ]:
im_data = mx.sym.Variable("im_data")
net_version = "lenet5_v2"
network = eval(net_version)(im_data)
mx.viz.plot_network(network,shape={"im_data":(128,1,28,28)},title=net_version)

# eval

In [1]:
import mxnet as mx
import os
import struct
import json
import numpy as np
import logging
import random
logging.getLogger().setLevel(logging.DEBUG)

In [2]:
def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,'{}-labels-idx1-ubyte'.format(kind))
    images_path = os.path.join(path,'{}-images-idx3-ubyte'.format(kind))
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II',
                                 lbpath.read(8))
        labels = np.fromfile(lbpath,
                             dtype=np.uint8).reshape(n)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack('>IIII',
                                               imgpath.read(16))
        images = np.fromfile(imgpath,
                             dtype=np.uint8).reshape((num,1,rows,cols))
    print(kind)
    print("label num:",n)
    print("image num:",num)
    print("image rows:",rows)
    print("image cols:",cols)
    images = images/255
    return images, labels

In [3]:
BACH_SIZE = 1000

In [4]:
val_im,val_la = load_mnist('/mnt/cephfs/lab/xuyan.bupt/dataset/mnist/','t10k')

t10k
label num: 10000
image num: 10000
image rows: 28
image cols: 28


In [5]:
val_iter = mx.io.NDArrayIter(data=val_im,
                             label=val_la,
                             batch_size=BACH_SIZE,
                            data_name='im_data',
                               label_name='softmax_label')

In [6]:
dev = [ mx.gpu(i) for i in [3]]

In [7]:
logs_dir = './logs/'
logs_names = os.listdir(logs_dir)

In [12]:
for name in logs_names:
    model_path = os.path.join(logs_dir,name,'lenet5-mnist')
    output_file = os.path.join(logs_dir,name,'score.json')
    json_info = {}
    for i in range(8):
        sym,arg_params,aux_params = mx.model.load_checkpoint(model_path,i+1)
        arg_name = sym.list_arguments()
        mod = mx.mod.Module(symbol=sym, 
                        context=dev,
                        data_names = [arg_name[0]],
                        label_names = [arg_name[-1]]
                       )
        mod.bind(for_training=False,
                 data_shapes=val_iter.provide_data,
                 label_shapes=val_iter.provide_label)
        mod.set_params(arg_params,aux_params)
        metric = mx.metric.Accuracy()
        score = mod.score(val_iter, metric)
        json_info[i+1] = score
    print(name,"===>")
    print(json.dumps(json_info))
#     f = open(output_file,'w')
#     f.write(json.dumps(json_info,indent=2))
#     f.close()

20181204T1011 ===>
{"1": [["accuracy", 0.9745]], "2": [["accuracy", 0.9825]], "3": [["accuracy", 0.9893]], "4": [["accuracy", 0.9892]], "5": [["accuracy", 0.9898]], "6": [["accuracy", 0.9897]], "7": [["accuracy", 0.99]], "8": [["accuracy", 0.9901]]}
20181204T0951 ===>
{"1": [["accuracy", 0.9689]], "2": [["accuracy", 0.9822]], "3": [["accuracy", 0.9845]], "4": [["accuracy", 0.985]], "5": [["accuracy", 0.9851]], "6": [["accuracy", 0.9884]], "7": [["accuracy", 0.989]], "8": [["accuracy", 0.9892]]}
